## SRJ - Simulating for NH3

In [3]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.0
symbols = ["N","H","H","H"]
electrons = 10  # Number of electrons
orbitals = 20  # Number of spin orbitals
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
active_electrons = 6  #3 from N and 3 from H
active_orbitals = 6  # Prince told me its spatial orbitals
core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals

H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals)
print('Qubits are', qubits)
print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))
hf_state = qchem.hf_state(active_electrons, qubits)





def adaptvqesri(adapt_it, e_th):
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                print('Exc. dealing right now is', ash_excitation[i])
                print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, 2*active_orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        # Start with the most recent state (last state in the list)
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        print('Moving towards parameters')
         #Parameters initialization
        params = np.append(params, 0.0)  #Parameters initialization
        print('The length of parameters is', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation




Qubits are 12
List of core orbitals: [0, 1]
List of active orbitals: [2, 3, 4, 5, 6, 7]
Number of qubits: 12


In [ ]:
adaptvqesri(adapt_it=50, e_th=1e-12)

Updated hf_state is [1 1 1 1 1 1 0 0 0 0 0 0]
HF state is -55.151336866435535
Total excitations are 117
The adapt iteration now is 0
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(0) a(8)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(8) is 1.4383587376709528e-10
The current excitation operator is a⁺(0) a(10)
Print, if this is activated - HF state


/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(0) a(10) is 8.037586824230503e-08
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(1) a(9)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(9) is 1.4383588764488309e-10
The current excitation operator is a⁺(1) a(11)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(11) is 8.037586825271337e-08
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 0.0
The current excitation operator is a⁺(2) a(8)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(8) is 1.9401459050438774e-09
The current excitation operator is a⁺(2) a(10)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(10) is 7.168938301327366e-08
The current excitation operator is a⁺(3) a(7)
Print, if this is activated - HF state
The exp

([-55.163587572817406,
  -55.16831512683961,
  -55.17316358556089,
  -55.17922811373288,
  -55.18139658326506,
  -55.182982716003316,
  -55.18673020258931,
  -55.189524186787324,
  -55.230642705259356,
  -55.23165201352718,
  -55.23275387706707,
  -55.23340647669748,
  -55.23410715638218,
  -55.23465511442056,
  -55.23513796891038,
  -55.23521813973973,
  -55.23529514144916,
  -55.23532819854188,
  -55.23533924366169,
  -55.23534891878965,
  -55.235387958850474,
  -55.23539754141273,
  -55.235425511472116,
  -55.235428243432864,
  -55.235430446140484],
 array([-0.25515429, -0.09871538, -0.09873496, -0.14666881, -0.05548995,
        -0.0544997 ,  0.10361164, -0.10373399, -1.5707962 ,  0.05531782,
         0.05531533,  0.04500489,  0.04501725, -0.03135044,  0.03135582,
         0.02049384, -0.0175728 , -0.00796814,  0.00418293, -0.00425081,
         0.02050836, -0.00802433, -0.01782235, -0.00246958,  0.00240832]),
 [[2, 3, 8, 9],
  [0, 3, 8, 11],
  [1, 2, 9, 10],
  [0, 1, 8, 9],
  [2, 3,

## HF comparison

In [11]:
## HF energy 
HF_P = -55.151336866435535
HF_FCI = -55.15133687573932

HF_P - HF_FCI

9.303782633196533e-09

In [5]:
-55.18139658326506 - -55.3187315806391

-55.235430446140484 - -55.3187315806391

0.08330113449861898

## FCI code for NH3 - active space

In [10]:
from pyscf import gto, scf, mcscf

# Define the molecule
mol = gto.M(
    atom='N 0.0 0.0 0.0; H 0.0 0.0 1.0; H 0.0 0.0 2.0; H 0.0 0.0 3.0;',
    basis='sto-3g',
    verbose=4  # Optional: Adjusts verbosity for output
)

# Perform Restricted Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()

# Get the HF energy
hf_energy = mf.e_tot
print("Hartree-Fock Ground State Energy:", hf_energy)

# Define the active space
ncas = 6  # Number of active orbitals - spatial it seems
nelecas = 6  # Number of electrons in the active space

# Set up the CASCI/CASSCF solver
mycas = mcscf.CASCI(mf, ncas, nelecas)

# Alternatively, use CASSCF for self-consistent optimization
# mycas = mcscf.CASSCF(mf, ncas, nelecas)

# Compute the CASCI/CASSCF energy
e_casci = mycas.kernel()

# Output the results
print("CASCI/CASSCF Ground State Energy:", e_casci)
print("Hartree-Fock Ground State Energy:", hf_energy)


System: uname_result(system='Darwin', node='chem177.chem.und.nodak.edu', release='23.6.0', version='Darwin Kernel Version 23.6.0: Fri Jul  5 17:53:24 PDT 2024; root:xnu-10063.141.1~2/RELEASE_ARM64_T6020', machine='arm64')  Threads 1
Python 3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:19:53) [Clang 18.1.8 ]
numpy 1.26.4  scipy 1.15.1  h5py 3.12.1
Date: Thu Mar 27 18:42:04 2025
PySCF version 2.8.0
PySCF path  /Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 N      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT] 